--------------Proje Raporu--------------

Öncelikle bu projede kaynak görevi gören log dosyasını aşağıdaki kodlar vasıtası ile oluşturdum.
Bu şekilde yapmamın nedeni bana verilen ödevde verileri kendimiz oluşturmamız gerektiği idi.


In [ ]:
import random
import pandas as pd
import datetime

# Function to generate random IP addresses
def generate_ip():
    return ".".join(str(random.randint(0, 255)) for _ in range(4))

# Function to generate random timestamps within a specific date range
def generate_timestamp(start, end):
    return start + (end - start) * random.random()

# Function to generate random HTTP methods
def generate_http_method():
    return random.choice(["GET", "POST", "DELETE", "PUT"])

# Function to generate random URL paths
def generate_url_path():
    categories = ["shirts", "pants", "jackets", "shoes", "accessories"]
    items = {
        "shirts": ["tshirt", "poloshirt", "dressshirt"],
        "pants": ["jeans", "shorts", "trousers", "bootcut"],
        "jackets": ["leather", "denim", "parka"],
        "shoes": ["sneakers", "boots", "sandals"],
        "accessories": ["hats", "scarves", "belts"]
    }
    category = random.choice(categories)
    item = random.choice(items[category])
    page = random.choice(["details", "reviews", "photos"])
    return f"/{category}/{item}/{page}"

# Function to generate random HTTP status codes
def generate_status_code():
    return random.choice([200, 301, 404, 500])

# Function to generate a log line
def generate_log_line():
    ip = generate_ip()
    timestamp = generate_timestamp(datetime.datetime(2023, 1, 1), datetime.datetime(2023, 12, 31)).strftime('%d/%b/%Y:%H:%M:%S')
    method = generate_http_method()
    url_path = generate_url_path()
    protocol = "HTTP/1.1"
    status_code = generate_status_code()
    return f'{ip} - - [{timestamp}] "{method} {url_path} {protocol}" {status_code} -'

# Generate 2000 log lines
log_lines = [generate_log_line() for _ in range(2000)]

# Create a DataFrame from log lines
df_logs = pd.DataFrame(log_lines, columns=["log"])

# Save to a CSV file
log_file_path = "/mnt/data/www_111mtk111_com_log.csv"
df_logs.to_csv(log_file_path, index=False)

# Show a portion of the dataframe to the user
df_logs.head()


Sonrasında elde ettiğim csv dosyası üzerinde düzenlemeler yaparak veriyi eğitime hazır
ve üzerinde çalışılabilir hale getirdim.
Başta tek bir log sütunu vardı ve veriler iç içe girmiş vaziyette idi. Regex ile sütunları parçalara ayırarak dataframe'i düzenledim.
Ardından timestamp sütunundaki ifadeleri datetime formatına çevirmek gerekiyordu. Bu işlem yapıldı.


In [ ]:
log_pattern = re.compile(r'(\S+) (\S+) (\S+) \[(.*?)\] "(.*?)" (\d{3}) (\S+)')


df[['ip', 'identity', 'user', 'timestamp', 'request', 'status_code', 'size']] = df['log'].str.extract(log_pattern)
df.drop(columns=['log', 'size'], inplace=True)


df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S')

Ardından vektörleştirme işlemi için tüm sütunları birleştirip bir bağlam oluşturdum.

In [ ]:
df['combined'] = df['request'] + " | Status Code: " + df['status_code'] + " | IP: " + df['ip'] + " | Timestamp: " + df['timestamp'].astype(str)

combined_embeddings = model.encode(df['combined'].tolist())

Sonrasında Sentence-BERT modelini yükleyerek vektörleştirmeye hazır hale getirdim ve ardından bu işlemi yaptım. Koddaki model dışında birkaç model daha denedim ancak son olarak bu modelde karar kıldım. Proje için daha uygun olduğu kanısındaydım.

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

combined_embeddings = model.encode(df['combined'].tolist())

Projenin Retrieval kısmı için bir FAISS index oluşturdum ve ardından vektörleri bu indexe ekledim.


In [ ]:
index = faiss.IndexFlatL2(combined_embeddings.shape[1])

index.add(np.array(combined_embeddings))

Kulanıcıdan doğal dilden soruları alabilmek için bir panel gerekli olduğu için basit bir panel oluşturdum.


In [ ]:
user_query = input("Sormak istediğiniz soruyu girin: ")

Kullanıcıdan alınan soruları vektörleştirme ve ardından FAISS indexinde en yakın sonuçlara ulaşulması ve bulunan log kayıtlarını DF'den çekme işlemleri yapıldı.

In [ ]:
query_embedding = model.encode([user_query])

D, I = index.search(query_embedding, k=5)

similar_logs = df.iloc[I[0]]
print(similar_logs)

Bu kısım daha önce tasarladığım sistemin sağlıklı sonuç vermemesi üzere eklediğim bir kısım. Soru- Cevap çiftleri halinde bulunan eğitim verileri oluşturularak modelin daha iyi eğitilmesi ve sistemin daha sağlıklı sonuç vermesi sağlanabilir. Bu kısımda sadece gösterme amaçlı yer verdim. Yeterli zamanım kalmamıştı.

In [ ]:
training_data = [
    {"question": "Kaç adet status_code değeri 200’dür?", "answer": "200 adet status_code bulunmaktadır."},
    {"question": "En çok erişilen sayfa nedir?", "answer": "GET /index.html"},
    # Diğer soru-cevap çiftleri buraya eklenebilir.
]

train_df = pd.DataFrame(training_data)

Sonrasında RAG modelinin jeneratif model kısmını kurmak için T5 seçildi. GPT seçmememin nedeni ise API'nın kullanım sıkıntısı oldu.

FAISS'den alınan verilerin kullanıcıya doğal dilde verilmesi gerektiği için metin verisini tokenlara yani sayısal dizilere dönüştürmek gerekti.

In [ ]:
model_name = "t5-small"
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

Gösterme amaçlı hazırladığım eğitim verilerinin T5 modelinin anlaması için aşağıdaki kod dizimini kullandım.

In [ ]:
def preprocess_data(data):
    input_texts = []
    target_texts = []
    for item in data:
        input_text = f"question: {item['question']} context: Log verileri"
        target_text = item['answer']
        input_texts.append(input_text)
        target_texts.append(target_text)
    return input_texts, target_texts

input_texts, target_texts = preprocess_data(training_data)

İnput ve Output metinlerini tokenize etme işlemi için gerekli kodlar yazıldı.

In [ ]:
train_encodings = t5_tokenizer(input_texts, truncation=True, padding=True, max_length=512)
train_labels = t5_tokenizer(target_texts, truncation=True, padding=True, max_length=512)


Aşağıdaki kod eğitim verisinin modelin anlayabileceği bir veri setine dönüştürür.

In [ ]:
train_dataset = {
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels["input_ids"]
}


Sistemi tasarlarken istediğim cevapları alamamam sonucunda daha özel bir eğitim yapmak amacı ile fine-tunning işlemini kullanmaya karar verdim. Gerekli kodları oluşturarak eğitim parametleri belirlenmiş oldu.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

Verilerin organize edilmesi ve modelin etkin bir şekilde eğitilmesi için gerekli kodlar yazıldı.

In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

Modelin fine-tunning sürecini başlatmak için gerekli olan veri setini ve eğitim yapılandırması sağlanması amacı ile gerekli kodlar oluşturuldu ve model fine-tune edildi.

In [ ]:
train_dataset = QADataset(train_encodings, train_labels["input_ids"])


trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=t5_tokenizer
)

trainer.train()

FAISS aramsı sonucunda elde ettiğmiz en yakın log kayıtlarını birleştirip metin formatında bir bağlam oluşturmamız gerekiyor idi. FAISS'ten dönen log kayıtları bağlama çevirildi ve sonrasında zengin bir bağlam oluşturuldu.

In [ ]:
log_text = " ".join(similar_logs['combined'].tolist())

input_text = f"question: {user_query} context: {log_text}"

Tokenize etme işlemi uygulanarak modelin işleyebileceği sayısal verilere dönüştürüldü.

In [ ]:
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

Modelin soru ve bağlamdan bir yanıt üretmesi sağlandı.

In [ ]:
outputs = t5_model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

Son olarak model tarafından oluşturulan sayısal çıktıyı yani tokenleri tekrar metin formatına çevirdim.

In [ ]:
answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Modelin cevabı: {answer}")

Model değerlendirmesi:

Model çok basit veya kolay sorulara cevap verebiliyor durumda olduğundan daha fazla geliştirilmesi gerekiyor. Özellikle RAG kısmındaki bilgi yetersizliğimden kaynaklı sorunlar yaşadım ancak biraz daha zaman ayırmam durumunda projeyi çok daha iyi yerlere götürebilirim.

Log dosyası gerçek verileri içermesi halinde çıkaracağım sonuçlar çok daha sağlıklı olacak ve anlamlı bir analiz yaparak modeli daha iyi eğitme fırsatım olacaktı. O yüzden gerçek bir log dosyasına erişmek de büyük önem taşıyor.

Kendi performansımı değerlendirecek olursam:

Elimden geleni yaptığımı düşünüyorum kesinlikle. Proje sağlıklı olarak çalışmasa da bir hayli yol kat etmiş durumda ve biraz daha süre verilmesi halinde kusursuza yakın bir sistem tasarlayabileceğim kanısındayım.

Son olarak ödevin yapım aşamasında çok fazla şey öğrendim ve bundan dolayı çok memnumum. Teşekkür ederim.